In [4]:
# preprocess
# copy each image into proper classification directory
import os
import numpy as np
from PIL import Image

path = "./sample/"
if os.path.exists(path):
    os.rmdir(path)
os.mkdir(path)
train_path = path + "train/"
if not os.path.exists(train_path):
    os.mkdir(train_path)
valid_path = path + "valid/"
if not os.path.exists(valid_path):
    os.mkdir(valid_path)
img_path = "../data/img/"

valid_p = 0.1

def saveimg(imgid, trg_dir, trg_shape=(224, 224)):
    if not os.path.exists(trg_dir):
        os.mkdir(trg_dir)
    src_img = img_path+"%s.jpg"%imgid
    trg_img = trg_dir+"%s.jpg"%imgid
    try:
        img=Image.open(src_img)
        small_img = img.resize(trg_shape)
        small_img.save(trg_img)
    except:
        print "saveimg %s failed"%src_img

# load attr file
img_attr_dict = {}
with open("../data/img_attr.csv", 'r') as fin:
    for line in fin:
        attrs = line.strip().split("\t")
        if len(attrs) != 7:
            continue
        imgid, zan_num, cai_num, show, hotness, date, title = attrs
        if date == "NAN":
            continue
        key = date[:7]
        img_attr_dict.setdefault(key, [])
        img_attr_dict[key].append((int(imgid), int(zan_num), int(cai_num), int(show), float(hotness)))

# label each image and copy to proper directory
target_key_set = set(["2015/01", "2015/02", "2015/03"])
label_id = 3 # show count
max_label = 4 
for key, img_attrs in img_attr_dict.items():
    if key not in target_key_set:
        continue
    img_attr_matrix = np.array(img_attrs)
    
    log_attrs = np.log(img_attr_matrix[:, label_id])
    max_value = log_attrs.max()
    min_value = log_attrs.min()
    for row_num in range(img_attr_matrix.shape[0]):
        attr = img_attr_matrix[row_num]
        
        imgid = int(attr[0])
        
        # label each img
        log_score = log_attrs[row_num]
        norm_score = (log_score-min_value)*1.0/(max_value-min_value+1)
        label = min(int(norm_score*10), max_label)
        
        if np.random.rand() > valid_p:
            # copy this img to train directory
            saveimg(str(imgid), train_path+str(label)+"/")
        else:
            # copy this img to valid directory
            saveimg(str(imgid), valid_path+str(label)+"/")

print "preprocess done!"

saveimg ../data/img/12671.jpg failed
saveimg ../data/img/16997.jpg failed
preprocess done!


In [16]:
np.log(158240)

11.971868146856538

In [17]:
np.log(1538)

7.3382381500655889